# Notebook for examining how ```geopy``` works

In [ ]:
!pip -q install geopy
!pip -q install folium

In [ ]:
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

from geopy.geocoders import Nominatim

import folium
from folium.plugins import FastMarkerCluster

from src.stedsans.nlp.ner.predict import NamedEntityRecognition

In [ ]:
d = NamedEntityRecognition().tag_sentence("Malte bor på Jægergårdsgade 23 Aarhus C")

In [ ]:
b_loc_idx = d.get("tag").index("B-LOC")
loc = d.get("word")[b_loc_idx]
print(loc)

In [ ]:
loc_list = []
counter = 0
for tag in d.get("tag"):
    if "LOC" in tag or "ORG" in tag:
        loc_list.append(d.get("word")[counter])
    counter += 1
    
loc_list

In [ ]:
len(d.get("tag"))

In [ ]:
d

In [ ]:
loc_list = []
for idx, tag in enumerate(d.get("tag")):
    if idx != len(d.get("tag")):
        if "B-LOC" in tag:
            loc_list.append(d.get("word")[idx])
        elif "I-LOC" in tag:
            loc_list.append(d.get("word")[idx])
        elif "B-ORG" in tag:
            loc_list.append(d.get("word")[idx])
        elif "I-ORG" in tag:
            loc_list.append(d.get("word")[idx])
loc_list            

In [ ]:
collapsed_result = []

entities = []         # Named entity instances will go here.
current_entity = []   # Tokens that are part of the current entity will go here.

last_tag = None       # We'll use this to check whether a token is part of the same entity as the previous.

for idx, tag in enumerate(d.get("tag")):

    token = d.get("word")[idx]
    
    if tag == 'O' or last_tag != tag:	# We've reached the end of the current entity.
    	# If that entity had a real tag (not 'O' or None), then save it.
        if last_tag != 'O' and last_tag != None:
        	# We save the list of tokens in this named entity, along with its tag, as a tuple.
            #if tag[0] == 
        	#  string.join() converts the list of tokens into a string.
            entities.append((' '.join(current_entity), last_tag))
        current_entity = []	# Reset for a new entity.
    last_tag = tag			# Keep track of the current entity tag; see lines 10 and 12.
    current_entity.append(token)

In [ ]:
entities

In [ ]:
collapsed_result = []

current_entity_tokens = []         # Named entity instances will go here.
current_entity = []   # Tokens that are part of the current entity will go here.

last_tag = None       # We'll use this to check whether a token is part of the same entity as the previous.

for idx, tag in enumerate(d.get("tag")):

    token = d.get("word")[idx]
    
    if tag == 'O':	# We've reached the end of the current entity.
        continue
    if tag.startswith("B-"):
         # ... if we have a previous entity in the buffer, store it in the result list
        if current_entity is not None:
            collapsed_result.append(
                (" ".join(current_entity_tokens), current_entity))

        current_entity = tag[2:]
        # The new entity has so far only one token
        current_entity_tokens = [token]
    # If the entity continues ...
    elif tag == "I-" + current_entity:
        # Just add the token buffer
        current_entity_tokens.append(token)
    else:
        raise ValueError("Invalid tag order.")

# The last entity is still in the buffer, so add it to the result
# ... but only if there were some entity at all
if current_entity is not None:
    collapsed_result.append(
        (" ".join(current_entity_tokens), current_entity))
    	

In [ ]:
def collapse(ner_result):
    # List with the result
    collapsed_result = []

    # Buffer for tokens belonging to the most recent entity
    current_entity_tokens = []
    current_entity = None

    # Iterate over the tagged tokens
    for token, tag in ner_result:
        if tag == "O":
            continue
        # If an enitity span starts ...
        if tag.startswith("B-"):
            # ... if we have a previous entity in the buffer, store it in the result list
            if current_entity is not None:
                collapsed_result.append(
                    (" ".join(current_entity_tokens), current_entity))

            current_entity = tag[2:]
            # The new entity has so far only one token
            current_entity_tokens = [token]
        # If the entity continues ...
        elif tag == "I-" + current_entity:
            # Just add the token buffer
            current_entity_tokens.append(token)
        else:
            raise ValueError("Invalid tag order.")

    # The last entity is still in the buffer, so add it to the result
    # ... but only if there were some entity at all
    if current_entity is not None:
        collapsed_result.append(
            (" ".join(current_entity_tokens), current_entity))
            
collapse(entities)



In [ ]:
entities = []         # Named entity instances will go here.
current_entity = []   # Tokens that are part of the current entity will go here.

last_tag = None       # We'll use this to check whether a token is part of the same entity as the previous.

for i in xrange(len(tagged_tokens)):    # Evaluate each token, in order.
	# Separate the token from its tag, so that we can evaluate them separately.
    token, tag = tagged_tokens[i]       

    if tag == 'O' or last_tag != tag:	# We've reached the end of the current entity.
    	# If that entity had a real tag (not 'O' or None), then save it.
        if last_tag != 'O' and last_tag != None:
        	# We save the list of tokens in this named entity, along with its tag, as a tuple.
        	#  string.join() converts the list of tokens into a string.
            entities.append((' '.join(current_entity), last_tag))
        current_entity = []	# Reset for a new entity.
    last_tag = tag			# Keep track of the current entity tag; see lines 10 and 12.
    current_entity.append(token)

In [ ]:
len(d.get("tag"))

In [ ]:
loc = " ".join(loc_list)
loc

In [ ]:
locator = Nominatim(user_agent="myGeocoder")
location = locator.geocode(loc)
print(location)

In [ ]:
lat = location.latitude
long = location.longitude
print(f"The coordinates for the lovely city of {loc} are: \nLatitude = {lat}, Longitude = {long}")

In [ ]:
map1 = folium.Map(
    location=[lat,long],
    tiles='cartodbpositron',
    zoom_start=15,
)

In [ ]:
map1

# FUCK SHIT I MADE IT WORK :D